# SUMMA forcing files
We need to convert the standard ERA5 forcing files (dimensions: lon, lat, time), into SUMMA-ready forcing files (dimensions: time, hru). 

Note: SUMMA expects the forcing file dimensions in the order (time, hru). See: https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#meteorological-forcing-files. Fortran and Python interact with netcdf dimensions in different ways and therefore the dimensions in this bit of Python code must be specified as (hru, time). This will ensure that Fortran interprets these in its desired order.

Explanation: 
This is a result of how multi-dimensional arrays are stored into memory in both languages. Fortran is `column-major`, whereas Python's `numpy` (which underlies the `netCDF4` library) is `row-major`. Note that this is completely **unrelated** to the way the multi-dimensional array is indexed, which is typically (row,column) in any programming language. See: https://en.wikipedia.org/wiki/Row-_and_column-major_order

In [1]:
# Modules
import sys # to handle command line arguments (sys.argv[0] = name of this file, sys.argv[1] = arg1, ...)
import netCDF4 as nc4
import numpy as np
from pathlib import Path # Load the path module to avoid windows/linux issues
#from simpledbf import Dbf5 # used to create a dataframe from .dbf file (Graham has issues with geopandas)
import geopandas as gpd # replace simpledbf

In [2]:
## General values
year = 1979
month = 1

In [3]:
# locations
path_att = Path('C:/Git_repos/summaWorkflow/2_MS_experiment_setup/NA_catchmentMeritHydro_ERA5_SOILGRIDS_MODISVEG_Graham/1_create_attributes/')
file_att = path_att / 'attributes_na_catchment_initial.nc' # for HRU order

In [4]:
# Location of the intersection shapefile, that tells us which bits of ERA5 to cut out for use with the NLDAS grid
path_intersection_shp = Path('C:/Git_repos/summaWorkflow/2_MS_experiment_setup/NA_catchmentMeritHydro_ERA5_SOILGRIDS_MODISVEG_Graham/4_create_forcing/intersect_output_cleaned')
file_intersection_shp = path_intersection_shp / 'intersect_era5_forcing_and_merit_hydro_basins.shp'

In [5]:
# Specify ERA5 source forcing file path and name
path_source = Path('C:/Globus endpoint')
name_source = 'ERA5_NA_' + str(year) + str(month).zfill(2) + '.nc'
file_source = path_source / name_source

In [6]:
# Specify target forcing file path and name
path_dest = Path('C:/Globus endpoint')
name_dest = 'ERA5_NA_gridEra5_SUMMA_' + str(year) + str(month).zfill(2) + '.nc'
file_dest = path_dest / name_dest

In [7]:
# Get HRU ID ordering in attributes.nc file
# ---------------------------------------------------------------
with nc4.Dataset(file_att) as att:
    IDs_att = att.variables['hruId'][:]

In [8]:
# Load the intersection shapefile and convert into a dataframe
# ---------------------------------------------------------------
shp_intersection = gpd.read_file(file_intersection_shp)

## CANDEX approach

In [9]:
# Relevant CANDEX code
# CANDEX modules
# ---------------------------------------------------------------
import glob
import time
#import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt

# additional packages for speed-up
import shapefile # PyShp library

In [10]:
# Specify the CANDEX case variable, indicating that source has a regular lat lon grid
case = 1

In [11]:
# Specify the dimension and variable names in the source file
time_dim = 'time' # name of the dimension time
lat_dim = 'latitude' # name of the dimension lat
lon_dim = 'longitude' # name of the dimension lon
lat_var_name = 'latitude' # name of the variable lat
lon_var_name = 'longitude' # name of the variable lon

# Define what we want to transfer straight from the source .nc file
transfer_var = ['airpres','LWRadAtm','SWRadAtm','pptrate','airtemp','spechum','windspd']
transfer_att = ['missing_value','units','long_name','standard_name']

In [12]:
with nc4.Dataset(file_source) as src:
    
    # Read the time data in the source file
    time_step = src.variables['time'][1:2] - src.variables['time'][0:1] # check the step size
    time_unit = src.variables['time'].units # units of the step size
        
    # We need the data step size in [s] for SUMMA
    # Find the conversion factor that takes [time_unit] to [s]
    if "second" in time_unit:
        time_conversion_factor = 1 # 1 second in a second
    elif "minute" in time_unit:
        time_conversion_factor = 60 # 60 seconds in a minute
    elif "hour" in time_unit:
        time_conversion_factor = 3600 # etc.
    elif "day" in time_unit:
        time_conversion_factor = 86400
    else:
        time_conversion_factor = -1 # fill value to indicate we couldn't find the time step size
        
    # Calculate the data_step [s]
    data_step = time_step[0] * time_conversion_factor # time_step is a list. use [0] to ensure data_step doesn't become a list
    print('Time step size is '+ str(time_step) + ' with units \'' + time_unit + '\'. data_step = ' + str(data_step) + 's.')

FileNotFoundError: [Errno 2] No such file or directory: b'C:\\Globus endpoint\\ERA5_NA_197901.nc'

In [13]:
# --- CANDEX functions
# netcdf reads
def read_value_lat_lon_nc(case,
                          lat_target, lon_target, name_of_nc,
                          name_of_variable, name_of_time_dim,
                          name_of_lat_var, name_of_lon_var,
                          name_of_lat_dim, name_of_lon_dim):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/candex
    @ author's email id:       sh.gharari@gmail.com
    @license:                  Apache2

    This function funcitons read different grids and sum them up based on the
    weight provided to aggregate them over a larger area
    
    Arguments
    ---------
    case: value [1,]
            1 is for 3-dimensional variable with 1-dimentional lat and lon
            2 is for 3-dimensional varibale with 2-dimentional lat and lon
            3 is for 2-dimensional variable with 1-dimentional lat and lon (time series)
    lat_target: lat value [1,]
    lon_target: lon value [1,]
    name_of_nc: full or part of nc file(s) name including nc, string, example 'XXX/*01*.nc'
    name_of_variable: name of the varibale, string
    name_of_time_dim: name of time dimension, string
    name_of_lat_var: name of lat variable, string
    name_of_lon_var: name of lon variable, string
    name_of_lat_dim: name of lat dimension, string
    name_of_lon_dim: name of lon dimension, string
    
    Returns
    -------
    data: a numpy array that has the read value of the NetCDF file for the lats, lons and weights
    """
    names_all = glob.glob(name_of_nc)
    names_all.sort()
    data = None

    # for to read on variouse nc files for target lat lon
    for names in names_all:
        
        da = xr.open_dataset(names, decode_times=False)
        
        # case 1, the varibale is 3-dimensional and lat and lon are one-dimnesional
        if case ==1:
            # finding the index for the lat for target_lat
            da_lat = da[name_of_lat_var] # reading the lat variable
            temp = np.array(abs(da_lat-lat_target)) # finding the distance to target_lat
            index_target_lat = np.array([temp.argmin()]) # finding the closest index to target_lat
            
            # finding the index for the lon for target_lon
            da_lon = da[name_of_lon_var] # reading the lon variable
            temp = np.array(abs(da_lon-lon_target)) # finding the distnace to target_lon
            index_target_lon = np.array([temp.argmin()]) # finding the closest index to target_lon
            
            # making sure that the lat and lon are only one value and not two
            index_target_lon = index_target_lon[0]
            index_target_lat = index_target_lat[0]
            
#             # porder of dimensions for the target variable
#             dataset = da[name_of_variable]
#             order_time_dim = dataset.dims.index(name_of_time_dim)
#             order_lat_dim = dataset.dims.index(name_of_lat_dim)
#             order_lon_dim = dataset.dims.index(name_of_lon_dim)
            
#             if order_time_dim == 0: # such as the varibaele dimension is time, lat, lon
#                 data_temp = dataset [:,index_target_lat,index_target_lon]
#             if order_time_dim == 2: # such as the varibaele dimension is lon, lat, time
#                 data_temp = dataset [index_target_lon,index_target_lat,:]
                
            
            # porder of dimensions for the target variable
            order_time_dim = da.variables[name_of_variable].dims.index(name_of_time_dim)
            order_lat_dim = da.variables[name_of_variable].dims.index(name_of_lat_dim)
            order_lon_dim = da.variables[name_of_variable].dims.index(name_of_lon_dim)
            
            if order_time_dim == 0: # such as the varibaele dimension is time, lat, lon
                data_temp = da[name_of_variable][:,index_target_lat,index_target_lon]
            if order_time_dim == 2: # such as the varibaele dimension is lon, lat, time
                data_temp = da[name_of_variable][index_target_lon,index_target_lat,:]

        # case 2, the varibale is 3-dimensional and lat and lon are 2-dimentional such as rotated lat lon
        if case ==2:
            # finding the index for the lat
            da_lat = da[name_of_lat_var]
            da_lon = da[name_of_lon_var]
            temp = np.array(abs(da_lat-lat_target)+abs(da_lon-lon_target))
            ind = np.unravel_index(np.argmin(temp, axis=None), temp.shape)
            ind = np.array(ind)
            
            # order of dimensions for the target variable
            dataset = da[name_of_variable]
            order_time_dim = dataset.dims.index(name_of_time_dim)
            order_lat_dim = dataset.dims.index(name_of_lat_dim)
            order_lon_dim = dataset.dims.index(name_of_lon_dim)
            
            if order_time_dim == 0: # such as the varibaele dimension is time, lat, lon
                index_target_lat = ind[0]
                index_target_lon = ind[1]
                data_temp = dataset [:,index_target_lat,index_target_lon]
            if order_time_dim == 2: # such as the varibaele dimension is lon, lat, time
                index_target_lat = ind[1]
                index_target_lon = ind[0]
                data_temp = dataset [index_target_lon,index_target_lat,:]

        # case 3, the varibale is 2-dimnesional and lat and lon are 1-dimensional such as n time or time n
        if case ==3:
            da_lat = da[name_of_lat_var]
            da_lon = da[name_of_lon_var]
            temp = np.array(abs(da_lat-lat_target)+abs(da_lon-lon_target))
            ind = np.unravel_index(np.argmin(temp, axis=None), temp.shape)
            ind = np.array(ind)
            ind = ind[0]
            
            # order of dimensions for the target variable
            dataset = da[name_of_variable]
            order_time_dim = dataset.dims.index(name_of_time_dim)
            
            if order_time_dim == 0: # such as the varibaele dimension is time, n
                index_target_n = ind
                data_temp = dataset [:,index_target_n]
            if order_time_dim == 1: # such as the varibaele dimension is n, time
                index_target_n = ind
                data_temp = dataset [index_target_n,:]
        
        # getting the length of time dimension
        time_steps = da.dims[name_of_time_dim]
        
        # put the read data into the data_temp
        data_temp = np.array(data_temp)
        data_temp = data_temp.reshape((time_steps, ))

        # append the data_temp
        if data is not None:
            data = np.append(data, data_temp)
        else:
            data = data_temp
            
    return data


# area weighting
def area_ave(case,
             lat, lon, w,
             name_of_nc, name_of_variable,
             name_of_time_dim,
             name_of_lat_dim, name_of_lon_dim,
             name_of_lat_var, name_of_lon_var):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/candex
    @ author's email id:       sh.gharari@gmail.com
    @license:                  Apache2

    This function funcitons read different grids and sum them up based on the
    weight provided to aggregate them over a larger area
    
    Arguments
    ---------
    case: value [1,]
            1 is for 3-dimensional variable with 1-dimentional lat and lon
            2 is for 3-dimensional varibale with 2-dimentional lat and lon
            3 is for 2-dimensional variable with 1-dimentional lat and lon (time series)
    lat: lat value [1,]
    lon: lon value [1,]
    w: wieght[1,]
    name_of_nc: full or part of nc file(s) name including nc, string, example 'XXX/*01*.nc'
    name_of_variable: name of the varibale, string
    name_of_time_dim: name of time dimension, string
    name_of_lat_dim: name of lat dimension, string
    name_of_lon_dim: name of lon dimension, string
    name_of_lat_var: name of lat variable, string
    name_of_lon_var: name of lon variable, string
    
    Returns
    -------
    data: a numpy array that has the read value of the NetCDF file for the lats, lons and weights
    """
    
    data = None 
    #print(w, lat, lon)
    if lat.size ==1: # only one entry to the funciton (one lat, one lon and one W)
        data_temp = read_value_lat_lon_nc(case,
                                          lat, lon, name_of_nc,
                                          name_of_variable, name_of_time_dim,
                                          name_of_lat_dim, name_of_lon_dim,
                                          name_of_lat_var, name_of_lon_var)
        data = data_temp * w
    else:
        for i in np.arange(lat.shape[0]):# itterate over target values
            data_temp = read_value_lat_lon_nc(case,
                                              lat[i], lon[i], name_of_nc,
                                              name_of_variable, name_of_time_dim,
                                              name_of_lat_dim, name_of_lon_dim,
                                              name_of_lat_var, name_of_lon_var)
            if i == 0: # multiply the read value with their weight and sum
                data = data_temp * w[i]
            else:
                data = data + data_temp * w[i]
    return data

In [ ]:
import time
start = time.time()

# Create the new netcdf file
with nc4.Dataset(file_dest, "w", format="NETCDF4") as dest, nc4.Dataset(file_source) as src:
    
    # === Some general attributes
    dest.setncattr('Author', "Created by W. Knoben from ERA5 source data")
    dest.setncattr('History','Created ' + time.ctime(time.time()))
    dest.setncattr('Source','Written using a script from the library of Wouter Knoben (Github link)')
    dest.setncattr('Reason','Takes (1) ERA5 data in lat/lon format for the North America domain, (2) a prepared shape file of the intersection between ERA5 grid and the NLDAS grid that we want to use, and creates SUMMA-ready (HRU-based) forcing.')
    
    # === Meta attributes from source
    for name in src.ncattrs():
        dest.setncattr(name, src.getncattr(name))
    
    # === Define the dimensions
    dest.createDimension('hru',len(IDs_att)) # use the number of HRU's defined in the attribute file
    dest.createDimension('time',len(src.dimensions['time'])) # source must have a 'time' dimension. Copy length of this dimension
    
    # === Define the HRU-specific topograhic variables: hruId, latitude, longitude
    # These values come from CANDEX. hruID is needed as one of the SUMMA dimensions (the other dimension is time).
    # lat/lon are the centers of the polygon in the case of gridded data. These are not used by SUMMA but kept in
    # case we want to use them to plot things later.
    var = 'hruId'
    dest.createVariable(var, 'i4', 'hru', fill_value = False)
    dest[var].setncattr('long_name', 'ID of the Hydrologic Response Unit')
    
    var = 'lat'
    dest.createVariable(var, 'f4', 'hru', fill_value = False)
    dest[var].setncattr('long_name', 'latitude')
    dest[var].setncattr('units', 'degrees_north')    
    
    var = 'lon'
    dest.createVariable(var, 'f4', 'hru', fill_value = False)
    dest[var].setncattr('long_name', 'longitude')
    dest[var].setncattr('units', 'degrees_east')    
    
    # === Create the non-HRU-specific variables: time and data_step
    var = 'data_step'
    dest.createVariable(var, 'f4', (), fill_value = False)
    dest[var].setncattr('long_name', 'Length of time step')
    dest[var].setncattr('units','seconds')
    dest.variables[var][:] = data_step
    
    var = 'time'
    dest.createVariable(var, src.variables[var].datatype, src.variables[var].dimensions, fill_value = -999)
    dest[var].setncatts(src.variables[var].__dict__)
    dest.variables[var][:] = src.variables[var][:]
    
    # === Define the HRU-specific topograhic variables: pptrate, LWRadAtm, SWRadAtm, airpres, airtemp, spechum, windspd
    # Copy these directly from the source file to ensure data provenance
    for name, variable in src.variables.items():
        
        # Transfer the forcing variables using the pre-defined list, using dimensions 'hru' and 'time'
        if name in transfer_var:
            dest.createVariable(name, variable.datatype, ('hru','time'), fill_value = -999)
            dest[name].setncatts(src[name].__dict__)
    
    # === Get the HRU ID's from the shapefile 'shp' on path 'path_shp'
    IDs = shp_intersection['hruId'].to_numpy().astype('int')
    
    # === Initialize a progress report
    # progress = 0
    
    # === Loop over HRU's and fill the variables
    for i in IDs_att:
        
        # Show a progress report
        # if (progress%100) == 0:
        #    print(str(progress) + ' out of ' + str(len(IDs)) + ' HRUs completed.')
        
        # We need to ensure that each HRU ID goes into the destination file in the right order:
        # Find the index of this particular HRU ID in attributes.nc
        idx = np.where(IDs_att == i)
        idx = list(idx[0]) # convert numpy array to list that we can use as index
        
        # Read the appropriate row(s) from the shapefile
        row = shp_intersection.loc[IDs == i]
        
        # Store topographic variables in the .nc file 
        # 'idx' reflects their position in the 'hru' dimension
        dest['hruId'][idx] = np.array(row['hruId'].iloc[0]) 
        #dest['lat'][idx] = np.array(row['hrulat'].iloc[0]) 
        #dest['lon'][idx] = np.array(row['hrulon'].iloc[0]) 
        
        # Read the lat, lon and weights
        lat = np.array(row.latForce) # lat of source
        lon = np.array(row.lonForce) # lon of source
        W = np.array(row.WForce) # contribution of source in targesh shape
        
        # Loop over forcing variables defined in 'transfer_var'
        for var in transfer_var:
            
            # Calculate the area weighted forcing
            # Note that the Pathlib path 'file_source' must be forced into a string for CANDEX to work
            awf = area_ave(case, lat, lon, W, str(file_source), var,\
                           time_dim, lat_dim, lon_dim, lat_var_name, lon_var_name )
            
            # Store this forcing variable in the .nc file 
            # 'idx' reflects its position in the 'hru' dimension
            dest[var][idx,:] = awf
            
# timer
end = time.time()
print('execution took' + str(end - start))

## Andy Wood's approach

In [56]:
dest.close()

In [ ]:
def compAvgVal(wgt, i_index, j_index, overlaps, f_dataIn, varname):
    """Compute areal weighted avg value of <varname> in <nc_in> for all output polygons
       based on input/output overlap weights in <nc_wgt>"""

    # now read the input timeseries file
    print("-------------------")
    print("reading input timeseries data for %s " % varname)
    dataVals = f_dataIn.variables[varname][:]    
    print("INFO: value at [0,0,0]: %f" % (dataVals[0,0,0]))

    matDataVals = np.zeros((nOutPolys, maxOverlaps))
    wgtedVals   = np.zeros((nTimeSteps, nOutPolys))

    # format data into shape matching weights [nOutPolys, maxOverlaps]
    for t in range(0, nTimeSteps):

        # format data into regular matrix matching format of weights (nOutPolygons, maxOverlaps)
        #   used advanced indexing to extract matching input grid indices
        for p in range(start_poly, (end_poly+1)):
            matDataVals[p, 0:overlaps[p]] = \
                dataVals[t, [j_index[overlapStartNdx[p]:overlapEndNdx[p]]], [i_index[overlapStartNdx[p]:overlapEndNdx[p]]] ] 
    
        wgtedVals[t, ] = np.sum(matDataVals * matWgts, axis=1)   # produces vector of weighted values
        print(" averaged var %s timestep %d" % (varname, t))

    return wgtedVals